# Here I need to translate OutSanomalies.m





In [7]:
using GLMakie,Makie

In [9]:
directory="/Users/nobuaki/Documents/MantleConvectionTakashi/toy_models_2D/"
file_header="test_bs" # basalt composition
# now we make an array of files with directory that start with file_header
#files = readdir(directory)  # get all files in the directory
files = filter(file -> startswith(file, file_header), readdir(directory)) ; # filter files that start with file_header



In [10]:
# test one file
file = directory*files[1]
scalardata=true
FileFormat = "n" # n-native, default, l-little endian, b-big endian
nval=0 # to initialise nval

# this is Julia version of ReadStag3D2.m (Boris Kaus, Paul Tackley +) until June 2021

if scalardata
    nval=1
    print("scalardata ok")
else
    nval=4
    print("scalardata ng")
end

# Read header
f=open(file,"r")
magic=read(f,UInt32)

if magic > 8000
    INTSTR=UInt64
    FLTSTR=Float64
    magic=magic-8000
    read(f,UInt32)
else
    INTSTR=UInt64
    FLTSTR=Float32
end
print(INTSTR)
println(magic)
if (magic<100 && nval>1) || (magic>300 && nval==1)
    error("wrong number of components in field")
end
magic = mod(magic, 100)
xyp=0
if magic >=9 && nval==4
    xyp = 1
end


nxtot=read(f,INTSTR)
nytot=read(f,INTSTR)
nztot=read(f,INTSTR)
nblocks=read(f,INTSTR) # 2 for Yinyang
aspect=Array{FLTSTR,1}(undef,2)
read!(f,aspect)
nnx=read(f,INTSTR)
nny=read(f,INTSTR)
nnz=read(f,INTSTR)
nnb=read(f,INTSTR)

nz2=nztot*2+1
zg=Array{FLTSTR,1}(undef,nz2)
read!(f,zg)

# compute nx, ny, nz and nb per CPU

nx=div(nxtot,nnx)
ny=div(nytot,nny)
nz=div(nztot,nnz)
nb=div(nblocks,nnb)
npi=(nx+xyp)*(ny+xyp)*nz*nb*nval

rcmb=read(f,FLTSTR)
istep=read(f,INTSTR)
time=read(f,FLTSTR)
erupta_total=read(f,FLTSTR)
botT_val=read(f,FLTSTR)
T_core = nothing
if magic >= 10
    T_core = read(f,FLTSTR)
end

x=Array{FLTSTR,1}(undef,nxtot)
y=Array{FLTSTR,1}(undef,nytot)
z=Array{FLTSTR,1}(undef,nztot)

# read the parallel blocks
DATA_3D = nothing
scalefac = nothing
VX_3D = nothing 
VY_3D = nothing
VZ_3D = nothing
P_3D = nothing # pressure data
if scalardata
    DATA_3D=zeros(FLTSTR,nxtot,nytot,nztot)
else
    scalefac=read(f,FLTSTR)
    VX_3D = zeros(FLTSTR,nxtot,nytot,nztot)
    VY_3D = zeros(FLTSTR,nxtot,nytot,nztot)
    VZ_3D = zeros(FLTSTR,nxtot,nytot,nztot)
    P_3D = zeros(FLTSTR,nxtot,nytot,nztot)
end
print(nxtot,",",nytot,",",nztot,"npi=",npi)
if scalardata # NF changed the if-clause out of 1000000000 for loops
    for ibc in 1:nnb
        for izc in 1:nnz
            for iyc in 1:nny
                for ixc in 1:nnx
                    data_CPU=Array{FLTSTR,1}(undef,npi) # read the data for 'this' CPU
                    read!(f,data_CPU)
                    # create a 3D matrix from these data

                    data_CPU_3D = reshape(data_CPU,(nx,ny,nz,nb))
                    
                    # add local 3D matrix to global matrix
                    
                    DATA_3D[(ixc-1)*nx+1:ixc*nx,(iyc-1)*ny+1:iyc*ny,(izc-1)*nz+1:izc*nz,(ibc-1)*nb+1,ibc*nb] = data_CPU_3D

                end
            end
        end
    end
else
    for ibc in 1:nnb
        for izc in 1:nnz
            for iyc in 1:nny
                for ixc in 1:nnx
                    data_CPU=Array{FLTSTR,1}(undef,npi) # read the data for 'this' CPU
                    read!(f,data_CPU)
                    # create a 3D matrix from these data
                    
                    data_CPU_3D = reshape(data_CPU*scalefac,(nval,nx+xyp,ny+xyp,nz,nb))
   
                    # add local 3D matrix to global matrix
                    
                    VX_3D[(ixc-1)*nx + (1:nx), (iyc-1)*ny +  (1:ny), (izc-1)*nz + (1:nz), (ibc-1)*nb + (1:nb)] = data_CPU_3D[1,1:nx,1:ny,:,:]
                    VY_3D[(ixc-1)*nx + (1:nx), (iyc-1)*ny +  (1:ny), (izc-1)*nz + (1:nz), (ibc-1)*nb + (1:nb)] = data_CPU_3D[2,1:nx,1:ny,:,:]
                    VZ_3D[(ixc-1)*nx + (1:nx), (iyc-1)*ny +  (1:ny), (izc-1)*nz + (1:nz), (ibc-1)*nb + (1:nb)] = data_CPU_3D[3,1:nx,1:ny,:,:]
                    P_3D[(ixc-1)*nx + (1:nx), (iyc-1)*ny +  (1:ny), (izc-1)*nz + (1:nz), (ibc-1)*nb + (1:nb)] = data_CPU_3D[4,1:nx,1:ny,:,:]

                end
            end
        end
    end
end



close(f)




scalardata okUInt6411
1,1024,128npi=4096

In [1]:
GLMakie.activate!()
#scene=Scene(backgroundcolor=:gray)
n=10
X,Y,Z=randn(n),randn(n),randn(n)
アスペクト=(1,1,1)
perspectiveness=0.5
ax=Axis3(; アスペクト,perspectiveness)
meshscatter!(ax,x,y,z;markersize=0.25)
#scene

UndefVarError: UndefVarError: `GLMakie` not defined

In [ ]:
function readStag3D(filename,attribute)


end 

In [21]:
w*0.1

3-element Vector{Float64}:
 0.1
 0.2
 0.30000000000000004